# Compute similarities of previous study for FGVC data.

Reproduce the results of [Measuring semantic similarity between concepts in visual domain](http://ieeexplore.ieee.org/document/4665152/).

In [1]:
%matplotlib inline

from scipy.misc import imread
import numpy as np
from skimage import color, filters, img_as_float
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import warnings
warnings.filterwarnings('ignore')

Define functions for getting tiles in an image and normalizing an image.

In [2]:
def get_tile(tile_row, tile_col, img, img_gray, img_binary):
    tile_h = int(img.shape[0]/5)
    tile_w = int(img.shape[1]/5)
    y0, y1, x0, x1 = tile_row*tile_h, ((tile_row+1)*tile_h), tile_col*tile_w, (tile_col+1)*tile_w
    tile = img[y0:y1, x0:x1, :]
    tile_gray = img_gray[y0:y1, x0:x1]
    tile_binary = img_binary[y0:y1, x0:x1]
    return tile, tile_gray, tile_binary

In [3]:
from scipy.misc import imresize
MAX_SIZE = 225

def im_normalized_read(fname, size_limit = MAX_SIZE):
    from skimage.transform import resize
    img =  imread(fname)
    max_len = np.max(img.shape)
    if max_len <= size_limit:
        return img_as_float(img)
    
    ratio = size_limit/max_len
    new_img = imresize(img, size=ratio, interp='bicubic')
    return img_as_float(new_img)

## Feature extraction

Here we treat four types of fetures: coloer moment, DOOG filter, orientation filter, shape features.

### Color moment

In [4]:
def calc_moment3(arr, moment1):
    cubed = np.mean((arr-moment1)**3)
    return math.pow(abs(cubed),1/3) * (1,-1)[cubed<0]

In [5]:
def calc_moments(arr):
    moment1 = np.mean(arr)
    moment2 = np.mean((arr-moment1)**2)**(1/2)
    moment3 = calc_moment3(arr, moment1)
    return moment1, moment2, moment3

In [6]:
def calc_4_rgb_moments(rgbarr):
    r1, r2, r3 = calc_moments(rgbarr[:, :, 0])
    g1, g2, g3 = calc_moments(rgbarr[:, :, 1])
    b1, b2, b3 = calc_moments(rgbarr[:, :, 2])
    gray = color.rgb2gray(rgbarr)
    gr1, gr2, gr3 = calc_moments(gray)
    return [r1, r2, r3, g1, g2, g3, b1, b2, b3, gr1, gr2, gr3]

### DOOG filter

Based on Figure 2 in [Preattentive texture discrimination with early vision mechanisms](https://www.ncbi.nlm.nih.gov/pubmed/2338600).

In [7]:
def G(y0, sigmax, sigmay, x, y):
    # The definition of G seems wrong, I refer original Yang's paper.
    # return (1/(2*math.pi* sigmax*sigmay)) *np.exp(- (x**2) + (y- y0/sigmay)**2)
    dimx = len(x)
    dimy = len(y)
    return (1/(2*math.pi* sigmax*sigmay)) *np.exp(- (1/2) * (((x/sigmax)**2).reshape(1, dimx) + (((y- y0)/sigmay)**2).reshape(dimy, 1)))

In [8]:
def DOOG2(x, y, sigma, r=3):
    sigmay = sigma
    sigmax = r*sigma
    ya = sigma
    yc = -sigma
    return -G(ya, sigmax, sigmay, x, y) + 2*G(0, sigmax, sigmay, x, y) - G(yc, sigmax, sigmay, x, y)

In [9]:
def DOOG2_weight(sigma, r=3, truncate=4):
    # similar filter size logic as filters.gaussian of scipy.
    lw = int(truncate * sigma + 0.5)
    return DOOG2(np.arange(-lw, lw+1), np.arange(-lw, lw+1), sigma, r)

In [10]:
# https://docs.scipy.org/doc/scipy-0.16.1/reference/generated/scipy.ndimage.filters.convolve.html
from scipy.ndimage.filters import convolve

In [11]:
def kernel2feature(img, weights):
    return np.mean([np.mean(convolve(img[:, :, i], weights)) for i in range(3)])

In [12]:
# use global variable for speed
# use sigma of 1, 2, 4, 8

KERNELS = [DOOG2_weight(i) for i in [1, 2, 4, 8]]

In [13]:
def tile2doog_features(tile):
    # to make scale similar
    return [1000*kernel2feature(tile, kernel) for kernel in KERNELS]

### Orientation feature

In [14]:
from scipy.stats import norm

In [15]:
def DOG(y, sigma1=0.5, sigma2=1.5):
    fy1 = norm.pdf(y, scale=sigma1)
    fy2 = norm.pdf(y, scale=sigma2)
    dog = fy1 - fy2
    return dog

In [16]:
def calc_f1(x, y, sigma):
    fx = norm.pdf(x, scale=sigma).reshape(1, len(x))
    fy = DOG(y).reshape(len(y), 1)
    # should access f1[y][x]
    return fx*fy

In [17]:
from scipy.signal import hilbert

In [18]:
def calc_f1f2(x, y, sigma):
    f1 = calc_f1(x, y, sigma)
    f2 = np.imag(hilbert(f1))
    return f1, f2

In [19]:
def rotation_indices(theta, shape):
    """return ([y1, y2, y3, ...], [x1, x2, x3, ...]). Use for numpy indices"""
    y_lim, x_lim = shape
    cos = math.cos(theta)
    sin = math.sin(theta)
    pairs = [(min(y_lim-1, max(0, int(x*sin + y*cos))),
              min(x_lim-1, max(0, int(x*cos - y*sin))))
            for y in range(y_lim)
            for x in range(x_lim)]
    return list(zip(*pairs))

Compute weights

In [20]:
def orientation_weights(theta, sigma, truncate=4):
    # similar filter size logic as filters.gaussian of scipy.
    lw = int(truncate * sigma + 0.5)
    
    x = np.arange(-lw, lw+1)
    y = np.arange(-lw, lw+1)
    f1, f2 = calc_f1f2(x, y, sigma)
    inds = rotation_indices(theta, f1.shape)
    f1r = f1[inds].reshape(f1.shape)
    f2r = f2[inds].reshape(f2.shape)
    return f1r, f2r

In [21]:
def orientation_energy(img, theta, sigma=1.5, truncate=4):
    f1, f2 = orientation_weights(theta, sigma)
    ex2 = convolve(img, f1)**2
    ey2 = convolve(img, f2)**2
    # make scale similar
    return 1000*np.mean(ex2+ey2)

In [22]:
def orientation_12_energy(gray_img):
    return [orientation_energy(gray_img, i*(2*math.pi)/12) for i in range(12)]

### Shape features

In [23]:
from skimage.filters import threshold_otsu

In [24]:
def gray2binary(gray_img):
    threds = threshold_otsu(gray_img)
    return gray_img <= threds

In [25]:
import skimage

#### Perimeter, area

In [26]:
# Paper says the ratio of the area to the perimeter squared, but inverse is much more common, which is compactness.
# standard definition divide this value with 4pi, but it's not important for our case.

def compactness(img_binary):
    l = skimage.measure.perimeter(img_binary)
    area = img_binary.sum()
    if area == 0:
        return 1, area
    return (l**2)/area, area

#### Moment of inertia

In [27]:
def calc_moment_of_inertia(gray_img):
    # cr, cc = ``M[1, 0] / M[0, 0]``, ``M[0, 1] / M[0, 0]`
    M1 = skimage.measure.moments(gray_img, order=1)
    cr = M1[1, 0]/M1[0, 0]
    cc = M1[0, 1]/M1[0, 0]
    mu2 = skimage.measure.moments_central(gray_img, cr, cc, order=2)
    # paper says shape feature is 4. So I use mu2_20 and mu2_02 as a separate features
    # return (mu2[2, 0]+mu2[0, 2])/mu2[0, 0]
    M0sq = mu2[0, 0]**2
    return mu2[2, 0]/M0sq, mu2[0, 2]/M0sq

In [28]:
from skimage.morphology import convex_hull_image
def ratio_convex_hull(img_bainary, area):
    hull_area = convex_hull_image(img_bainary).sum()
    return area/hull_area

#### Combine all shape features

In [29]:
def shape_features(img_gray, img_binary):
    comp, area = compactness(img_binary)
    mom = calc_moment_of_inertia(img_gray)
    if(img_binary.any()):
        ratio3 = ratio_convex_hull(img_binary, area)
    else:
        ratio3 = 1
    return [comp, mom[0], mom[1], ratio3]

### Combine all of features for a given tile

Define a function that combine all of featuers defined above.

In [30]:
def tile2features(tile, tile_gray, tile_binary):
    rgb_moments12 = calc_4_rgb_moments(tile)
    doog4 = tile2doog_features(tile)
    orientation12 = orientation_12_energy(tile_gray)
    shape4 = shape_features(tile_gray, tile_binary)
    return [elem for arr in [rgb_moments12, doog4, orientation12, shape4]
            for elem in arr]

### Define functions for extracting features for a given image.

A given image is divided into patches and features are extracted from each patch.  
Finally we concatenate those features into one feature vector to express characteristics of a given image.

In [31]:
def get_tile_features(tile_row, tile_col, img, img_gray, img_binary):
    tile, tile_gray, tile_binary = get_tile(tile_row, tile_col, img, img_gray, img_binary)
    return tile2features(tile, tile_gray, tile_binary)

In [32]:
def img2features(img):
    img_gray = color.rgb2gray(img)
    img_binary = gray2binary(img_gray)
    featuresList = [get_tile_features(row, col, img, img_gray, img_binary) for row in range(5)
                    for col in range(5)]
    return [f for fs in featuresList
               for f in fs]

## Extract features and save them into dataframe

In [33]:
from models.modelutils import dir2filedict_sorted

Using TensorFlow backend.


In [34]:
fdict1 = dir2filedict_sorted("data_fgvc/train")
fdict2 = dir2filedict_sorted("data_fgvc/valid")
fdict3 = dir2filedict_sorted("data_fgvc/test")

In [35]:
files = sorted(f for fdict in [fdict1, fdict2, fdict3] for key in fdict.keys() for f in fdict[key])

In [36]:
len(files)

10000

In [37]:
machine_id=0
# machine_id=1
# machine_id=2
# machine_id=3
# machine_id=4
# machine_id=5
# machine_id=6
# machine_id=7

In [38]:
target_files = files[(machine_id*1250):(machine_id*1250+1250)]

In [39]:
import pandas as pd

In [40]:
failed = []
flist = []

In [41]:
import tqdm

In [42]:
%%time

for file in tqdm.tqdm(target_files):
    try:
        current = file
        features = img2features(im_normalized_read(file))
        flist.append(features)
    except:
        print("fail {}".format(file))
        failed.append(file)

100%|██████████| 1250/1250 [6:16:38<00:00, 18.08s/it]  

CPU times: user 5h 36min 45s, sys: 38min 33s, total: 6h 15min 19s
Wall time: 6h 16min 38s


In [43]:
print( "Total number of images: {}".format(len(flist)) )
print( "The number of failed images from where features cannot be extracted: {}".format(len(failed)) )

Total number of images: 1250
The number of failed images from where features cannot be extracted: 0


Check images from which features cannot be extracted.

In [44]:
failed

[]

Pick up images from which features can be successfully extracted,

In [45]:
files2 = [f for f in target_files if f not in failed]
len(files2)

1250

In [46]:
df = pd.DataFrame(flist)
df.shape

(1250, 800)

In [47]:
df['filepath'] = files2

Save the extracted features as pickle.

In [48]:
df.to_pickle("results/features_{}_fgvc.dat".format(machine_id))

In [49]:
df.head()["filepath"]

0    data_fgvc/test/0/0062765.jpg
1    data_fgvc/test/0/0064932.jpg
2    data_fgvc/test/0/0197342.jpg
3    data_fgvc/test/0/0447936.jpg
4    data_fgvc/test/0/0536515.jpg
Name: filepath, dtype: object

In [51]:
files[0:5]

['data_fgvc/test/0/0062765.jpg',
 'data_fgvc/test/0/0064932.jpg',
 'data_fgvc/test/0/0197342.jpg',
 'data_fgvc/test/0/0447936.jpg',
 'data_fgvc/test/0/0536515.jpg']

### Concat 8 splited df

In [52]:
dflist = [pd.read_pickle("results/features_{}_fgvc.dat".format(machine_id)) for machine_id in range(0, 8)]

In [53]:
len(dflist)

8

In [54]:
all_df = pd.concat(dflist)

In [56]:
all_df.shape

(10000, 801)

In [64]:
df = all_df

## Load extracted features and estimate GMM

In [1]:
import pandas as pd
import numpy as np
import math

In [72]:
def path_to_cat(fpath):
    return fpath.split("/")[2]

In [74]:
df['filepath'].iloc[5000]

'data_fgvc/train/33/2233490.jpg'

In [73]:
path_to_cat(df['filepath'].iloc[5000])

'33'

In [75]:
cats = sorted(list(set(map(path_to_cat, df['filepath']))))

In [79]:
all((str(cat) in cats) for cat in range(0, 100))

True

In [80]:
len(cats)

100

In [81]:
df['category'] = list(map(path_to_cat, df['filepath']))

### For empty tile, moment will be na; fill na as 0 to avoid errors

In [82]:
df_fixed = df.fillna(0)

### Fit GMM for each class

We use the same 25 components GMM as that of the previsou study.

In [83]:
from sklearn.mixture import GaussianMixture

In [84]:
num_components = 25

In [85]:
def cat_to_model(catindex):
    onecat_df = df_fixed[df_fixed['category'] == cats[catindex]]
    onecat_fs = onecat_df.iloc[:, 0:800].as_matrix()
    
    # Section 4 in the paper denotes they assume N(mu, sigma) type.
    gmm = GaussianMixture(num_components, covariance_type = "diag")
    gmm.fit(onecat_fs)
    return gmm

In [86]:
%%time
models = [cat_to_model(ind) for ind in range(len(cats))]

CPU times: user 10.1 s, sys: 35.9 s, total: 46 s
Wall time: 5.8 s


Collect (means, variances) and weights for each class.

In [87]:
gmm_stats = [(mod.means_, mod.covariances_) for mod in models]

In [88]:
gmm_weights = [mod.weights_ for mod in models]

## Compute distances between classes using obtrained GMM.

Define functions for computing distances between components of GMM.

In [89]:
def calc_one_dij(mean_list1, sigma_list1, mean_list2, sigma_list2, i, j):
    return math.sqrt(((mean_list1[i] - mean_list2[j])**2+(sigma_list1[i] - sigma_list2[j])**2).sum())

In [90]:
def calc_dij(mean_list1, sigma_list1, mean_list2, sigma_list2):
    return np.array([calc_one_dij(mean_list1, sigma_list1, mean_list2, sigma_list2, i, j)
              for i in range(num_components)
              for j in range(num_components)]).reshape(num_components, num_components)

Here we use linear programming to obtrain distances.

Assume $w_{ij}$ is ordered like $w_{00}, w_{01}, w_{02}, w_{03}, ... w_{024}, w_{10}, w_{11}, ...$

In [91]:
from scipy.optimize import linprog

$\sum_i  w_{ij} = \beta_j$.

In [92]:
def sum_i_coeff(target_j):
    return [float(j == target_j) for i in range(num_components) for j in range(num_components)]

In [93]:
coeff_i_sums = [sum_i_coeff(j) for j in range(num_components)]

$\sum_j w_{ij} = \alpha_i$.

In [94]:
def sum_j_coeff(target_i):
    return [float(i == target_i) for i in range(num_components) for j in range(num_components)]

In [95]:
coeff_j_sums = [sum_j_coeff(i) for i in range(num_components)]

Take summation of these quantities. 

In [96]:
A_eq = coeff_i_sums+coeff_j_sums

Define a function that computes distances between classes.

In [97]:
def calc_distance(midx1, midx2):
    alpha = gmm_weights[midx1]
    beta = gmm_weights[midx2]
    mus1, sigs1 = gmm_stats[midx1]
    mus2, sigs2 = gmm_stats[midx2]
    dij =  calc_dij(mus1, sigs1, mus2, sigs2)

    b_eq = np.concatenate((beta, alpha))
    c = dij.reshape(num_components*num_components)
    res = linprog(c, A_eq=A_eq, b_eq=b_eq,
              options={"disp": False})
    return res.fun, res

Check the function.

In [98]:
dist, _ = calc_distance(3, 5)
dist

1882.2386117073297

In [99]:
dist, _ = calc_distance(5, 3)
dist

1882.2386117073302

Define a function that computes all of combinations of classes.

In [100]:
def calc_all_distances():
    catnum = len(cats)
    dists = np.zeros((catnum, catnum))
    for i in range(catnum):
        for j in range(catnum):
            if i != j :
                dists[i, j], _ = calc_distance(i, j)
    return dists

In [101]:
%%time
Dps = calc_all_distances()

CPU times: user 43min 7s, sys: 224 ms, total: 43min 8s
Wall time: 43min 8s


In [102]:
distdict = {cat1:[Dps[idx1, idx2] for idx2 in range(len(cats))] for idx1, cat1 in enumerate(cats)}

In [103]:
distdf = pd.DataFrame(distdict)

In [104]:
distdf.index = cats

In [105]:
distdf

,0,1,10,11,12,13,14,15,16,17,...,90,91,92,93,94,95,96,97,98,99
0,0.000000,3046.101723,2766.734586,2711.677687,2688.777579,2743.839675,2590.393673,2594.210519,2650.326974,2577.656687,...,2714.402983,2997.000191,3076.719676,2781.475849,2561.215999,3022.085364,2672.066689,2588.475212,2605.386746,2629.264591
1,3046.101723,0.000000,2849.662796,2708.251458,2801.636500,2899.499029,2770.364251,2756.692692,2884.996360,2529.276559,...,2839.954682,2738.126168,2855.586494,2875.307947,2666.155282,3231.025934,2902.864155,2845.743584,2601.261617,2643.561652
10,2766.734586,2849.662796,0.000000,2494.856044,2423.844458,2504.021387,2524.001962,2526.126148,2469.383571,2359.287047,...,2550.577044,2750.634276,2919.823783,2574.946701,2355.566715,3046.951759,2529.105031,2451.530760,2293.018897,2382.422271
11,2711.677687,2708.251458,2494.856044,0.000000,1819.472431,1882.238612,1981.044374,2268.803090,1933.103852,1938.998622,...,2388.277646,2456.423287,2632.645813,2052.422769,1945.523153,2778.166754,2393.411670,2262.840573,1711.385774,1800.233268
12,2688.777579,2801.636500,2423.844458,1819.472431,0.000000,1571.298240,1677.635451,2074.505230,1763.373174,1766.016224,...,2265.443490,2662.699749,2713.646650,1954.495641,2013.024223,2748.078514,2349.454300,2172.470655,1603.973938,1628.980478
13,2743.839675,2899.499029,2504.021387,1882.238612,1571.298240,0.000000,1574.243284,2074.371073,1664.210507,1943.158989,...,2227.308114,2624.011412,2699.208131,1913.604924,2030.158165,2806.539494,2384.066569,2325.459227,1654.500816,1671.549527
14,2590.393673,2770.364251,2524.001962,1981.044374,1677.635451,1574.243284,0.000000,2107.960979,1794.162352,1881.548371,...,2312.282799,2689.437264,2652.330247,2060.374043,2149.752301,2875.740337,2345.694282,2361.136180,1758.322312,1807.040761
15,2594.210519,2756.692692,2526.126148,2268.803090,2074.505230,2074.371073,2107.960979,0.000000,2187.504876,2084.234127,...,2460.621170,2746.283514,2824.654106,2233.598131,2236.976402,2842.645679,2386.256780,2295.463715,2029.110138,2059.619056
16,2650.326974,2884.996360,2469.383571,1933.103852,1763.373174,1664.210507,1794.162352,2187.504876,0.000000,1923.838422,...,2283.056251,2490.262057,2622.898013,2020.908875,1975.970734,2805.010469,2436.915758,2230.436956,1808.108301,1764.165039
17,2577.656687,2529.276559,2359.287047,1938.998622,1766.016224,1943.158989,1881.548371,2084.234127,1923.838422,0.000000,...,2317.635029,2563.783606,2613.069840,2006.881975,1983.574764,2761.378480,2169.330812,2239.756052,1745.089306,1720.713349


Save the results as pickle.

In [106]:
distdf.to_pickle("results/GMMDisttances_fgvc.dat")

## Show distances for each target class

In [46]:
# def show_cat_dist(catidx):
#     print(cats[catidx])
#     print(distdf[cats[catidx]].sort_values())

In [47]:
# [show_cat_dist(i) for i in range(len(cats))]